### Use one of the following datasets to perform sentiment analysis on the given Amazon reviews. 

### Pick one of the "small" datasets that is a reasonable size for your computer. 

### The goal is to create a model to algorithmically predict if a review is positive or negative just based on its text. Try to see how these reviews compare across categories. Does a review classification model for one category work for another?*

### Questions that need answering:

 1. What question are you trying to solve (or prove wrong) ?   
 __We are trying to predict the sentiment of the review.__
 1. What kind of data do you have? -> describe the source.. 
 __Review data from Amazon music,  This includes the rating, from 1-5, and the review comments.__
 1. Do some EDA, plots  
 __Since this is sentiment analysis, we really do not have any EDA.__. 
 1. What's missing from the data and how do you deal with it?  
 __There are no missing elements from this dataset, and minimal cleanup.  __
 1. Where are the outliers and why should we pay attention to them?  
 __We do not have any outliers here.__  
 1. Why did you pick this model for the project?
 I picked logistic regression, a classification model, with a binary outcome, favorable or non-favorable, as that is exactly what we are trying to do here; sentiment analysis.__
 1. How can you add, change, or remove features to get more out of your data? 
 __Not a lot of things I can think of to do in a sentiment analysis model.__
Items that could be done inclue: Feature importances, PCA, dropping low variance items, correlated feature pairs, features too highly correlated, feature engineering, transforming, timestamp and make a month, day, year column.

In [1035]:
url = 'http://jmcauley.ucsd.edu/data/amazon/'

In [1036]:
# Constants
max_iterations         = 10            # set it to > 0 for determining the features inportance
random_state           = 57
rows_in_training_set   = 10000
rows_in_test_set       = 200000
test_size              = 0.20
train_size             = 0.80c
rfc_test_size          = 50000
rfc_train_size         = 5000
run_CountVectorizer    = True
run_TfidfVectorizer    = False
BegTimeStampNewlines   = 3
EndTimeStampNewlines   = 3
EndTimeStamp           = '\n'*EndTimeStampNewlines+'End'
BegTimeStamp           = 'End'+'\n'*BegTimeStampNewlines

SyntaxError: invalid syntax (<ipython-input-1036-722c008479c7>, line 7)

In [1037]:
# Controls
flag_to_run_rfc = False
flag_to_run_rfr = False
flag_to_plot_them = False
flag_to_run_correlation_matrix = False
flag_to_run_features_importance = False
flag_to_run_gradient_boosting  = False
flag_to_run_linear_regression  = False
flag_to_run_logistic_regression = False
flag_to_run_sentiment_analyis = False
flag_to_run_svc = False
flag_to_run_vectorizer_nb = True

In [1038]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt

%matplotlib inline

import datetime
from sklearn import ensemble
from sklearn import datasets
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import time, sys
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from IPython.display import Markdown, display

In [1039]:
# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [1040]:
file = 'reviews_Digital_Music_5.json'
path = path="../../../../Datafiles/"
df = pd.read_json(path+file, lines=True)
columns_to_skip = ['reviewerName']
print("there are {:,} entries in the data file {}".format(len(df), file))

there are 64,706 entries in the data file reviews_Digital_Music_5.json


In [1041]:
# df.columns
print(df[['overall','reviewText']].sample(10))

       overall                                         reviewText
23491        4  My first acquaintance with Aimee Mann is this ...
44060        5  This 2-disc best of collection from the Eagles...
34995        4  Is this a great album? By all means! Is it Mil...
60324        4  Born This Way is an odd record that blends ele...
34659        4  This is stainds best album to date and the who...
42940        5  In a time where most popular hip hop has gone ...
52732        5  just keep putting them out and believe u me im...
18480        4  Volcano is not as consistently strong as some ...
45390        5  This album was my first introduction to Sufjan...
39316        5  I know I'm a little late on reviewing this alb...


In [1042]:
# data Cleanup
columns_to_cleanup = []
for column in columns_to_cleanup:
    print("we are now cleaning up column {}".format(column))
    df[column].fillna(-1, inplace=True)
# columns_to_use = []
# print(columns_to_use)
df['message'] = df['reviewText']
df['sentiment'] = df['overall']

# convert to a numerical variable.  
# set only the 4's and 5's to 1, which is positive, and all others to 0.
df['sentiment_label'] = df.sentiment.map({4:1, 5: 1, 3:0, 2: 0, 1:0}) 

print('we have cleaned up the dataframe.')

we have cleaned up the dataframe.


In [1043]:
df.sample(10)

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,message,sentiment,sentiment_label
16095,B000002L4I,"[0, 0]",4,Carlene Carter first appeared in the end of th...,"12 11, 2012",A3URRZ02P8KLWM,Joe Kuether,Unique character,1355184000,Carlene Carter first appeared in the end of th...,4,1
44970,B00016XMOE,"[1, 3]",4,I could want to listen the following three son...,"09 7, 2004",AFV9SYG84ET9I,"Nihit Saxena ""nihits""",Only three songs,1094515200,I could want to listen the following three son...,4,1
27652,B00000FDHE,"[1, 9]",2,"DJ Quik is a great producer, but this album do...","02 8, 2005",A1PULJNO1T3YVX,The Best Rap Critic,2.5 Stars,1107820800,"DJ Quik is a great producer, but this album do...",2,0
52794,B000GG4XI8,"[0, 0]",5,"This is a great CD! Now I was born in 1966, bu...","09 13, 2012",A9GYC5DZSSSLT,Amazon Customer,The time machine!,1347494400,"This is a great CD! Now I was born in 1966, bu...",5,1
42728,B0000A5A0K,"[1, 3]",3,Hmmm. BT seems to have drifted right into car...,"11 24, 2004",A363QRXP83K8ZE,"N. P. Stathoulopoulos ""nick9155""",Vroooooom,1101254400,Hmmm. BT seems to have drifted right into car...,3,0
1342,B000000NN1,"[14, 15]",5,"It is a well-know fact that Yes-frontman, Jon ...","12 21, 2002",A1IANEBSMVGHS9,"Manny Hernandez ""@askmanny""",The music he could have recorded with Yes...,1040428800,"It is a well-know fact that Yes-frontman, Jon ...",5,1
11722,B000002GF0,"[0, 1]",5,A great disc from the Brothers Johnson. My fa...,"12 20, 2006",A36NUDST4Y5JBA,"D. S. HARDEN ""a movie fan""",Light up the Night,1166572800,A great disc from the Brothers Johnson. My fa...,5,1
41146,B000089RVR,"[3, 3]",4,Dido's voice has this ethereal feminity to it ...,"05 6, 2007",A18MBO1U4DPY20,Harkanwar Anand,Make up your heart to break it again,1178409600,Dido's voice has this ethereal feminity to it ...,4,1
23289,B000003N94,"[9, 21]",2,John Coltrane may be the most influential and ...,"11 25, 2000",A6FIAB28IS79,Samuel Chell,Head for the exits.,975110400,John Coltrane may be the most influential and ...,2,0
55434,B000XXWK9I,"[2, 3]",3,Rick Ross was never great but he had the poten...,"03 25, 2008",A8SCX6VUTE05H,"Nuisance ""the rebel""",Better than his first album but not by much,1206403200,Rick Ross was never great but he had the poten...,3,0


In [1044]:
def print_timestamp(displaytext):    
    import sys
    import datetime
    datetime_now = str(datetime.datetime.now())
    print("{:19.19}: In: {} {} ".format(datetime_now, sys._getframe(1).f_code.co_name, displaytext))

In [1045]:
df.isnull().sum()

asin                 0
helpful              0
overall              0
reviewText           0
reviewTime           0
reviewerID           0
reviewerName       177
summary              0
unixReviewTime       0
message              0
sentiment            0
sentiment_label      0
dtype: int64

In [1046]:
df.sentiment.unique()

array([5, 4, 3, 1, 2])

In [1047]:
# Definine outcome and predictors.
print("the current time of start is {}".format(str(datetime.datetime.now())))

y = df['sentiment']
X = df['message']

the current time of start is 2019-06-09 15:16:36.861644


In [1048]:
X.head(5)

0    It's hard to believe "Memory of Trees" came ou...
1    A clasically-styled and introverted album, Mem...
2    I never thought Enya would reach the sublime h...
3    This is the third review of an irish album I w...
4    Enya, despite being a successful recording art...
Name: message, dtype: object

In [1049]:
def printFormatted(string):
    display(Markdown(string))

In [1050]:
def plot_them():
    for column in X_train.columns:
        plt.hist(X_train[column]*100, bins=40)
        plt.xlabel(column)
        plt.show()

In [1051]:
def rfc_and_feature_importances(leaf_values):    # Here we are using Gradient Boosting classifier method to determine the top 30 features.
# train and then run RFC
    
    params = {'n_estimators': 500
              ,'max_depth'    : 2
             }

    rfc = ensemble.RandomForestClassifier(**params)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=rfc_test_size, train_size=rfc_train_size)
    
    ## Fit the model on your training data.
    rfc.fit(X_train, y_train) 
    
    ## And score it on your testing data.
    rfc.score(X_test, y_test)

    feature_importance = rfc.feature_importances_

    # Make importances relative to max importance.
    feature_importance = 100.0 * (feature_importance / feature_importance.max())
    sorted_idx = np.argsort(feature_importance)
    cols=X.columns[sorted_idx].tolist() 
    cols=cols[::-1]
    pos = np.arange(sorted_idx.shape[0]) + .5
    plt.subplot(1, 2, 2)
    plt.barh(pos, feature_importance[sorted_idx], align='center')
    plt.yticks(pos, X.columns[sorted_idx])
    plt.xlabel('Relative Importance')
    plt.title('Variable Importance')
    plt.show()
    print("We are returning these columns {}".format(cols))
    return cols # return it sorted

In [1052]:
def rf_and_feature_importances(rf_type, leaf_values):    # Here we are using Gradient Boosting classifier method to determine the top 30 features.
# train and then run RFC
    
    params = {'n_estimators': 500
              ,'max_depth'    : 2
             }

    if rf_type == 'classifier':
        rf = ensemble.RandomForestClassifier(**params)
    elif rf_type == 'regression':
        rf = ensemble.RandomForestRegressir(**params)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=rfc_test_size, train_size=rfc_train_size)
    
    ## Fit the model on your training data.
    rf.fit(X_train, y_train) 
    
    ## And score it on your testing data.
    rf.score(X_test, y_test)

    feature_importance = rfc.feature_importances_

    # Make importances relative to max importance.
    feature_importance = 100.0 * (feature_importance / feature_importance.max())
    sorted_idx = np.argsort(feature_importance)
    cols=X.columns[sorted_idx].tolist() 
    cols=cols[::-1]
    pos = np.arange(sorted_idx.shape[0]) + .5
    plt.subplot(1, 2, 2)
    plt.barh(pos, feature_importance[sorted_idx], align='center')
    plt.yticks(pos, X.columns[sorted_idx])
    plt.xlabel('Relative Importance')
    plt.title('Variable Importance')
    plt.show()
    print("We are returning these columns {}".format(cols))
    return cols # return it sorted

In [1053]:
def run_feature_importances(type):
# Here we will return the feature importances
    all_feature_important_columns = []
 
    for i in range(1,max_iterations):
        print_timestamp('running rfc iteration {} features importance for {} times'.format(i,max_iterations))
        columns2 = rf_and_feature_importances(type, i)
#         columns2.extend('{}'.format(i))
        all_feature_important_columns = all_feature_important_columns + columns2
    #     print("all_feature_import_columns={}".format(all_feature_important_columns))

    print("\nBOD:\nall_feature_important_columns = {}\nEOD".format(sorted(all_feature_important_columns)))
    for feature in set(all_feature_important_columns):
        print("the number of occurrences of feature {} in all_feature_important_columns is {}".format(feature, all_feature_important_columns.count(feature)))

In [1054]:
def run_correlation_matrix():
    
    print_timestamp('Begin')
    
    # Setup the correlation matrix.
    corrmat = X.corr()
    print(corrmat)

    # Set up the subplots
    f, ax = plt.subplots(figsize=(12, 9))

    # Let's draw the heatmap using seaborn.
    sns.heatmap(corrmat, vmax=.6, square=True)
    plt.show()
    
    print_timestamp('End')

In [1055]:
# Let's fit it with the RFC training set
if (len(X) > 0):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, train_size=train_size, random_state=0)
    print("train_size = {}, X_train is {}, and y_train is {}".format(train_size, len(X_train), len(y_train)))
    print("test_size  = {}, X_test  is {}, and y_test is {}".format(test_size, len(y_train), len(y_test)))

train_size = 0.8, X_train is 51764, and y_train is 51764
test_size  = 0.2, X_test  is 51764, and y_test is 12942


In [1056]:
def run_rf(func):

    print_timestamp('Begin fit_train_rf')
    
    ## Fit the model on your training data.
    func.fit(X_train, y_train) 
    
    #Let's run cross validate score with the training data set
    cross_val_score(func, X_train, y_train, cv=5)
        
    ## Let's score it with the training data set
    fun .score(X_train, y_train)
    
    ## Let's score it with the test data set
    func.score(X_test, y_test)
    
    print_timestamp('End run_rf')

In [1057]:
def run_rf(func):
    print_timestamp('Begin run_rfr part 1')
    
    ## Fit the model on your training data.
    func.fit(X_train, y_train) 
    
    #Let's run cross validate score with the training data set
    cross_val_score(func, X_train, y_train, cv=5)
    
#     ## Let's score it with the training data set
#     rfr.score(X_train, y_train)
    
#     ## Let's score it with the test data set
#     rfr.score(X_test, y_test)
    
    print_timestamp('End run_rfr part 1')

In [1058]:
if run_rf == True:
    print_timestamp('Begin run_rfr part 2')
    
#     ## Fit the model on your training data.
#     rfr.fit(X_train, y_train) 
    
#     #Let's run cross validate score with the training data set
#     cross_val_score(rfr, X_train, y_train, cv=5)
    
    ## Let's score it with the training data set
    rfr.score(X_train, y_train)
    
    ## Let's score it with the test data set
    rfr.score(X_test, y_test)
    
    print_timestamp('End run_rfr part 2')

Let's try predicting with gradient boosting classification

In [1059]:
def run_gradient_boosting():

    print_timestamp('Begin')
    
    clf = ensemble.GradientBoostingClassifier(**params)

    #Let's run cross validate score with the training data set
    cross_val_score(clf, X_train, y_train, cv=5)

    loss_function = 'deviance' # could be exponential
    depth_value = 8
    params = {'n_estimators': 500,
              'max_depth': 8,
              'loss': loss_function,
              'max_leaf_nodes': depth_value, # 8 worked best...
              'min_samples_leaf': depth_value * 3
              ,'random_state' : random_state
             }

    clf.fit(X_train, y_train)

    predict_train = clf.predict(X_train)
    predict_test = clf.predict(X_test)
    
    print_timestamp('End')

In [1060]:
def run_svc():

    print_timestamp('Begin')
    
    # Let's do a linear Support Vector Classifier
    print_timestamp('Running SVC(kernel=linear')
    svm = SVC(kernel = 'linear')
    
    # Let's fit the training model
    print_timestamp('Running svm.fit')
    svm.fit(X_train, y_train)
    
    # Let's score the training set
    print_timestamp('Running svm.score for the training set')
    svm.score(X_train, y_train)
    
    # Let's score the test set
    print_timestamp('Running svm.fit for the test set')
    svm.score(X_test, y_test)

    print_timestamp('End')

In [1061]:
def run_logistic_regression():
    print_timestamp('Begin')

    lr = LogisticRegression(C=1e20, solver='lbfgs', max_iter=1000)

    print_timestamp('Running lr.fit for the training set')
    lr.fit(X_train, y_train)
    
    print_timestamp('Running lr.fit for the training set')
    print('\nR-squared simple model training set yields:')
    print(lr.score(X_train, y_train))
    print("here comes the test set")
    print(lr.score(X_test, y_test))
    
    print_timestamp('End')

In [1062]:
def run_linear_regression():

    print_timestamp('Begin run_linear_regression')
    
    regr = linear_model.LinearRegression()

    print_timestamp('Running regr.fit for the training set')
    regr.fit(X_train, y_train)
    
    print("\nCoeffecients: \n", regr.coef_)
    print("\nIntercept: \n", regr.intercept_)
    print("\nR-squared for training data set:")
    print(regr.score(X_train, y_train))
    
    print("\nR-squared for test data set:")
    print(regr.score(X_test, y_test))
    
    print_timestamp('End run_linear_regression')

In [1063]:
def run_bnb():
    print_timestamp('Begin run_bnb')

    bnb = BernoulliNB()

    print_timestamp('Running bnb.fit for the training set')
    bnb.fit(X_train, y_train)
    
    y_pred = bnb.predict(d1, t1)
    
    print_timestamp('Running lr.fit for the training set')
    print('\nR-squared simple model training set yields:')
    print(lr.score(X_train, y_train))
    print("here comes the test set")
    print(lr.score(X_test, y_test))
    
    print_timestamp('End')

In [1064]:
def run_Bernoulli_supervised_learning(d1, t1):
# Our data is binary / boolean, so we're importing the Bernoulli classifier.

    # Instantiate our model and store it in a new variable.
    bnb = BernoulliNB()

    # Fit our model to the data.
#     bnb.fit(d1, target)
    bnb.fit(X_train, y_train)


    # Classify, storing the result in a new variable.
    y_pred = bnb.predict(X_train)

    # Display our results.
    return_message = "Number of mislabeled points out of a total {} points : {}".format(
        d1.shape[0],
        (t1 != y_pred).sum()
    )

    return (return_message, d1.shape[0], (t1 != y_pred).sum(), (t1 != y_pred).sum()/d1.shape[0] )

In [1065]:
# X.columns

In [1066]:
def get_file_encoding(fn):
    import chardet
        
    with open(fn, 'rb') as f:
        content = f.read()

    charset = chardet.detect(content)
    # {'encoding': 'EUC-JP', 'confidence': 0.99}
    #print("character set = {}".format(charset['encoding']))
    
    return charset['encoding']

In [1067]:
# Let's process the data)
def open_and_load_file (filename, columnnames):
    file_encoding = get_file_encoding(filename)
    df = pd.read_csv(filename, delimiter= '\t', header=None, encoding=file_encoding) 
    df.columns = columnnames
    return df, file_encoding

In [1068]:
def run_Bernoulli_supervised_learning(d1, t1):
# Our data is binary / boolean, so we're importing the Bernoulli classifier.
    from sklearn.naive_bayes import BernoulliNB

    # Instantiate our model and store it in a new variable.
    bnb = BernoulliNB()

    # Fit our model to the data.
    bnb.fit(d1, target)

    # Classify, storing the result in a new variable.
    y_pred = bnb.predict(d1)

    # Display our results.
    return_message = "Number of mislabeled points out of a total {} points : {}".format(
        d1.shape[0],
        (t1 != y_pred).sum()
    )

    return (return_message, d1.shape[0], (t1 != y_pred).sum(), (t1 != y_pred).sum()/d1.shape[0] )

In [1069]:
def do_the_keywords():
    for key in keywords:
        # Add spaces around the key so that we are getting theword,
        # not just pattern matching
        sentiment_raw[str(key)] = sentiment_raw.message.str.contains(' ' + str(key) + ' ', case=False)

In [1070]:
def do_the_heatmap(subplotno):
    plt.subplot(3,1,subplotno) 
    sns.heatmap(sentiment_raw.corr(), cmap="Blues")

In [1071]:
def set_data_and_target():
    global data 
    data = sentiment_raw[keywords]
    # data.sample(5)
    
    global target 
    target = sentiment_raw['sentiment']
    # target.head(10)

In [1072]:
def vectorizer_nb():

    print_timestamp(BegTimeStamp)
    
    # 1. import and instantiate CountVectorizer (with the default parameters)

    # 2. instantiate CountVectorizer (vectorizer)

    X = df.message
    y = df.sentiment_label

    # split X and y into training and testing sets
    # by default, it splits 75% training and 25% test
    # random_state=1 for reproducibility
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

    # 3. fit
    print("run_CountVectorizer={}, and run_TfidfVectorizer={}".format(run_CountVectorizer,
                                                                      run_TfidfVectorizer))
    if run_CountVectorizer == True:
        print("We are running with CountVectorizer")
        vectorizer = CountVectorizer()
        vectorizer.fit(X_train)
        vectorizer_method = 'CountVectorizer'
    elif run_TfidfVectorizer == True:
        print("We are running with TfidfVectorizer")
        vectorizer = TfidfVectorizer()
        vectorizer.fit_transform(X_train)
        vectorizer_method = 'TfidfVectorizer'
    
#     vectorizer_fit(X_train)

    # 4. transform training data
    X_train_dtm = vectorizer.transform(X_train)

    # equivalently: combine fit and transform into a single step
    # this is faster and what most people would do
    X_train_dtm = vectorizer.fit_transform(X_train)

    # 4. transform testing data (using fitted vocabulary) into a document-term matrix
    X_test_dtm = vectorizer.transform(X_test)
    X_test_dtm

    # you can see that the number of columns, 7456, is the same as what we have learned above in X_train_dtm

    # 1. import

    # 2. instantiate a Multinomial Naive Bayes model
    nb = MultinomialNB()

    # 3. train the model 
    # using X_train_dtm (timing it with an IPython "magic command")

    nb.fit(X_train_dtm, y_train)
    
    # 4. make class predictions for X_test_dtm
    y_pred_class = nb.predict(X_test_dtm)

    # calculate accuracy of class predictions

    met_score = metrics.accuracy_score(y_test, y_pred_class)
    printFormatted('###  With {} vectorizer, and Multinomial Naive Bayes,  the metrics accuracy score = {:.2%}'.format(vectorizer_method,
                                                                                         met_score))
    
    print_timestamp(EndTimeStamp)

In [1073]:
rfc = ensemble.RandomForestClassifier()
rfr = ensemble.RandomForestRegressor()

In [1074]:
if flag_to_plot_them == True:
    plot_them()
    
if flag_to_run_features_importance == True:
    run_features_imporantance(rfc)
    
if flag_to_run_correlation_matrix == True:
    run_correlation_matrix()
    
if flag_to_run_rfc == True:
    run_rfc()
        
if flag_to_run_rfr == True:
    run_rfr()
    
if flag_to_run_gradient_boosting  == True:
    run_gradient_boosting()
    
if flag_to_run_linear_regression  == True:
    run_linear_regression()
    
if flag_to_run_logistic_regression == True:
    run_logistic_regression()
    
if flag_to_run_svc == True:
    run_svc() 
    
if flag_to_run_vectorizer_nb == True:
    vectorizer_nb()

2019-06-09 15:16:37: In: vectorizer_nb End


 
run_CountVectorizer=True, and run_TfidfVectorizer=False
We are running with CountVectorizer


###  With CountVectorizer vectorizer, and Multinomial Naive Bayes,  the metrics accuracy score = 82.98%

2019-06-09 15:16:59: In: vectorizer_nb 


End 


### Post Model Analysis:  

#### As it turns out, the most effective model for this project was with CountVectorization, and MulitNomial Naive Bayes.  I attempted to use TfidfVectorization and MultiNomial Bayes, but it yielded a lower accuracy score of more the 1 percent.